### 실시간 인트라데이 데이터 수집
 - 5분 간격의 실시간 주가 데이터
 - AAPL, GOOGL, TSLA 종목 데이터
 - 시간대별 거래량 통계
 
### 기술적 지표 데이터
 - RSI (Relative Strength Index) 지표
 - 재사용 가능한 함수 형태로 구현
 - 지표별 통계 분석
 
### 데이터 관리
 - 타임스탬프가 포함된 파일명으로 저장
 - API 호출 제한 준수
 - 오류 처리 및 데이터 검증

In [1]:
# Alpha Vantage 실시간 인트라데이 주가 데이터
import os
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

#API_KEY = os.environ.get('ALPHAVANTAGE_API_KEY')
API_KEY = 'OACKEXPDGZWGF7KE'
BASE = 'https://www.alphavantage.co/query'

# 실시간 인트라데이 데이터 설정
symbols = ['AAPL', 'GOOGL', 'TSLA']
params_template = {
    'function': 'TIME_SERIES_INTRADAY',
    'interval': '5min',  # 1min, 5min, 15min, 30min, 60min 중 선택
    'outputsize': 'compact',  # compact(최근 100개) 또는 full
    'datatype': 'json'
}

os.makedirs('data/collected', exist_ok=True)


In [2]:
# 실시간 인트라데이 데이터 수집
frames = []
if API_KEY:
    for sym in symbols:
        try:
            params = params_template | {'symbol': sym, 'apikey': API_KEY}
            r = requests.get(BASE, params=params, timeout=30)
            r.raise_for_status()
            js = r.json()

            # API 응답 확인
            time_series_key = f'Time Series ({params["interval"]})'
            ts = js.get(time_series_key, {})

            if not ts:
                print(f"{sym}: 데이터 없음")
                continue

            # DataFrame 생성
            df = pd.DataFrame(ts).T.reset_index().rename(columns={'index': 'timestamp'})

            # 데이터 타입 변환
            numeric_cols = ['1. open', '2. high', '3. low', '4. close', '5. volume']
            for col in numeric_cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')

            df['symbol'] = sym
            df['interval'] = params['interval']
            frames.append(df)

            # API 호출 제한 준수 (초당 5회, 분당 500회)
            time.sleep(0.25)

        except Exception as e:
            print(f"{sym}: 오류 발생 - {str(e)}")
            continue

intraday_df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()


In [3]:
# 최근 5개 데이터 확인
if not intraday_df.empty:
    print(f"총 데이터 개수: {len(intraday_df)}")
    print("\n최근 5개 데이터:")
    display(intraday_df.head())

    # 시간대별 통계
    intraday_df['datetime'] = pd.to_datetime(intraday_df['timestamp'])
    intraday_df['hour'] = intraday_df['datetime'].dt.hour

    print("\n시간대별 평균 거래량:")
    hourly_volume = intraday_df.groupby('hour')['5. volume'].mean()
    display(hourly_volume)
else:
    print("데이터가 없습니다 (API_KEY 확인 필요)")


총 데이터 개수: 300

최근 5개 데이터:


timestamp   1. open  2. high  3. low  4. close  5. volume symbol  \
0  2025-09-19 19:55:00  245.6600   245.70  245.65  245.6900      11669   AAPL   
1  2025-09-19 19:50:00  245.6900   245.70  245.63  245.6400       5513   AAPL   
2  2025-09-19 19:45:00  245.6900   245.74  245.66  245.6600       2654   AAPL   
3  2025-09-19 19:40:00  245.6901   245.74  245.66  245.6900       2525   AAPL   
4  2025-09-19 19:35:00  245.6700   245.75  245.65  245.6942       4239   AAPL   

  interval  
0     5min  
1     5min  
2     5min  
3     5min  
4     5min


시간대별 평균 거래량:


hour
11    5.280088e+05
12    7.028065e+05
13    4.021595e+05
14    5.564586e+05
15    9.783682e+05
16    5.279983e+06
17    2.101183e+04
18    1.541553e+04
19    6.667056e+03
Name: 5. volume, dtype: float64

In [4]:
# CSV 파일로 저장
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = f'data/collected/alpha_vantage_intraday_{timestamp}.csv'

if not intraday_df.empty:
    # 시간 순서대로 정렬
    intraday_df = intraday_df.sort_values('timestamp', ascending=False)
    intraday_df.to_csv(output_file, index=False)
    print(f"저장 완료: {output_file}")
else:
    print("저장할 데이터가 없습니다")


저장 완료: data/collected/alpha_vantage_intraday_20250921_202234.csv


In [5]:
# Alpha Vantage 기술적 지표 데이터 (RSI)
def get_technical_indicator(symbol, function='RSI', interval='daily', time_period=14):
    """기술적 지표 데이터 가져오기"""
    params = {
        'function': f'{function}',
        'symbol': symbol,
        'interval': interval,
        'time_period': time_period,
        'series_type': 'close',
        'datatype': 'json',
        'apikey': API_KEY
    }

    try:
        r = requests.get(BASE, params=params, timeout=30)
        r.raise_for_status()
        js = r.json()

        # Technical Analysis 키 찾기
        tech_key = f'Technical Analysis: {function}'
        ts = js.get(tech_key, {})

        if ts:
            df = pd.DataFrame(ts).T.reset_index().rename(columns={'index': 'date'})
            df['symbol'] = symbol
            df['indicator'] = function
            return df
    except Exception as e:
        print(f"{symbol} {function}: 오류 - {str(e)}")

    return pd.DataFrame()

# RSI 지표 데이터 수집 예시
rsi_frames = []
for sym in symbols[:2]:  # 상위 2개 종목만
    df = get_technical_indicator(sym, 'RSI', 'daily', 14)
    if not df.empty:
        rsi_frames.append(df)
    time.sleep(0.25)  # API 제한 준수

rsi_df = pd.concat(rsi_frames, ignore_index=True) if rsi_frames else pd.DataFrame()


In [6]:
# RSI 데이터 확인
if not rsi_df.empty:
    print("RSI 데이터 (최근 10개):")
    display(rsi_df.head(10))

    # RSI를 숫자로 변환
    rsi_df['RSI'] = pd.to_numeric(rsi_df['RSI'], errors='coerce')

    print("\nRSI 통계:")
    print(rsi_df.groupby('symbol')['RSI'].describe())
else:
    print("RSI 데이터가 없습니다")


RSI 데이터 (최근 10개):


date      RSI symbol indicator
0  2025-09-19  67.9316   AAPL       RSI
1  2025-09-18  60.4466   AAPL       RSI
2  2025-09-17  62.4172   AAPL       RSI
3  2025-09-16  61.5361   AAPL       RSI
4  2025-09-15  60.0341   AAPL       RSI
5  2025-09-12  57.2207   AAPL       RSI
6  2025-09-11  52.4456   AAPL       RSI
7  2025-09-10  48.1344   AAPL       RSI
8  2025-09-09  59.9006   AAPL       RSI
9  2025-09-08  67.0019   AAPL       RSI


RSI 통계:
         count       mean        std      min        25%      50%        75%  \
symbol                                                                         
AAPL    6497.0  54.851495  12.967665  17.0723  45.238000  54.7262  64.205300   
GOOGL   5292.0  54.373686  12.548834  21.3305  45.120525  54.3614  63.357275   

            max  
symbol           
AAPL    91.5037  
GOOGL   90.9293  
